In [15]:
import os
import csv
import json
from time import sleep
from spotipy import Spotify
from spotipy.util import prompt_for_user_token
from spotipy.oauth2 import SpotifyClientCredentials

os.environ['DISPLAY'] = ':1'

In [16]:
credentials = json.load(open('credentials', 'rt'))

In [79]:
def follow_artist(id, max_degree=None, degree=0, from_id=None, from_weight=None):
    global sp, token, artists, artists_writer, artists_csvfile, relationships_writer, relationships_csvfile
    while True:
        try:
            artist = sp.artist(id)
            break
        except KeyboardInterrupt:
            raise
        except:
            sleep(2)
            token = prompt_for_user_token('teticio', '', credentials['spotify_client_id'], credentials['spotify_client_secret'], credentials['spotify_redirect_uri'])
            sp = Spotify(token)
            continue
    id = artist['id']
    weight = int(artist['popularity'])/100
    if id not in artists:
        artists_writer.writerow({'id:ID': id, 'name': artist['name'].replace('"', "'"), 'popularity:float': weight, 'url': artist['external_urls']['spotify'], ':LABEL': 'ARTIST'})
        artists_csvfile.flush()
    if from_id:
        relationships_writer.writerow({':START_ID': from_id, 'weight:float': round(weight + from_weight, 4), ':END_ID': id, ':TYPE': 'RELATED_TO'})
        relationships_csvfile.flush()

    if id in artists:
        return # I've been here before
    artists[id] = artist['name']
    print(f'{len(artists)} ({degree}). {artists[id]} {artist["external_urls"]["spotify"]}')
    if max_degree and degree >= max_degree:
        return
    while True:
        try:
            related_artists = sp.artist_related_artists(id)
            break
        except KeyboardInterrupt:
            raise
        except:
            sleep(2)
            token = prompt_for_user_token('teticio', '', credentials['spotify_client_id'], credentials['spotify_client_secret'], credentials['spotify_redirect_uri'])
            sp = Spotify(token)
            continue
    for artist in related_artists['artists']:
        follow_artist(artist['id'], max_degree, degree + 1, from_id=id, from_weight=weight)

In [80]:
artists = {}

artists_csvfile = open('spotify-artists-header.csv', 'wt', newline='')
fieldnames = ['id:ID', 'name', 'popularity:float', 'url', ':LABEL']
artists_writer = csv.DictWriter(artists_csvfile, fieldnames=fieldnames, quoting=csv.QUOTE_NONNUMERIC)
artists_writer.writeheader()
artists_csvfile.close()
artists_csvfile = open('spotify-artists.csv', 'wt', newline='')
artists_writer = csv.DictWriter(artists_csvfile, fieldnames=fieldnames, quoting=csv.QUOTE_NONNUMERIC)

relationships_csvfile = open('spotify-relationships-header.csv', 'wt', newline='')
fieldnames=[':START_ID', 'weight:float', ':END_ID', ':TYPE']
relationships_writer = csv.DictWriter(relationships_csvfile, fieldnames=fieldnames, quoting=csv.QUOTE_NONNUMERIC)
relationships_writer.writeheader()
relationships_csvfile.close()
relationships_csvfile = open('spotify-relationships.csv', 'wt', newline='')
relationships_writer = csv.DictWriter(relationships_csvfile, fieldnames=fieldnames, quoting=csv.QUOTE_NONNUMERIC)

In [81]:
#sp = spotipy.Spotify(SpotifyClientCredentials(credentials['spotify_client_id'], credentials['spotify_client_secret']))
token = prompt_for_user_token('teticio', '', credentials['spotify_client_id'], credentials['spotify_client_secret'], credentials['spotify_redirect_uri'])
sp = Spotify(token)

In [82]:
[(_['artists'][0]['name'], _['artists'][0]['id']) for _ in sp.search('artist:Klang')['tracks']['items']]

[('Klangkarussell', '041iTeoMIwXMlShuQPIVKo'),
 ('Klangkarussell', '041iTeoMIwXMlShuQPIVKo'),
 ('Klangkarussell', '041iTeoMIwXMlShuQPIVKo'),
 ('Klangkarussell', '041iTeoMIwXMlShuQPIVKo'),
 ('Klangkarussell', '041iTeoMIwXMlShuQPIVKo'),
 ('Klangkarussell', '041iTeoMIwXMlShuQPIVKo'),
 ('Klangkarussell', '041iTeoMIwXMlShuQPIVKo'),
 ('Klangkarussell', '041iTeoMIwXMlShuQPIVKo'),
 ('Klangkuenstler', '6H77vD9YyhyxHBTkRpbMBk'),
 ('Klangspiel', '2IM5tvnq8r2d8yYY2FTXNX')]

In [83]:
follow_artist('041iTeoMIwXMlShuQPIVKo', 5)

1 (0). Klangkarussell https://open.spotify.com/artist/041iTeoMIwXMlShuQPIVKo
2 (1). Bakermat https://open.spotify.com/artist/3MyFDtqB80WZvbtCZRsekM
3 (2). GAMPER & DADONI https://open.spotify.com/artist/6HQ6vf4AloXyVNdyJhrX1J
4 (3). SRTW https://open.spotify.com/artist/7vHAcPVlEDksRsCfkez2CD
5 (4). Y.V.E. 48 https://open.spotify.com/artist/5zSWGyWE5d0PYaYrtdVwOz
6 (5). Alex Schulz https://open.spotify.com/artist/5OReTeTBZ578sj07hDS0x7
7 (5). Max Manie https://open.spotify.com/artist/0esUZwk3FcZiAH1fXa66dU
8 (5). Möwe https://open.spotify.com/artist/4S8NmgM7oJ188sKp1waZpy
9 (5). De Hofnar https://open.spotify.com/artist/1LFGVkNRmXz8jIkyPjaGNL
10 (5). Sons Of Maria https://open.spotify.com/artist/2SfhiNVhk8w8xCQ89P3NPU
11 (5). Nicolas Haelg https://open.spotify.com/artist/0nwid3SFCGyXxnllTHzHPd
12 (5). Vijay & Sofia Zlatko https://open.spotify.com/artist/23BtKJexFLn0Ri6s7BEC4d
13 (5). Dimmi https://open.spotify.com/artist/7HIN94LvSDgxN5RoOLi3E5
14 (5). Deepend https://open.spotify.com/ar

114 (5). Jubël https://open.spotify.com/artist/4FcZfItjVIsfO9TynErl7X
115 (5). PANG! https://open.spotify.com/artist/6TlN75Ns66CyxBuZ7i3TK3
116 (3). Deep Chills https://open.spotify.com/artist/12rBrcOPP50qIan8ew4iTP
117 (4). Axero https://open.spotify.com/artist/1f3NYrlm9GpXxMz8XZGEP5
118 (5). Vexento https://open.spotify.com/artist/3tYJfCciy07wTBt2HpjCQw
119 (5). Diviners https://open.spotify.com/artist/22lnnGKlaDxk8sfzCNRJuA
120 (5). Janji https://open.spotify.com/artist/5VXPJC0PNpJbzB10A39G8E
121 (5). Tobu https://open.spotify.com/artist/1feoGrmmD8QmNqtK2Gdwy8
122 (5). Itro https://open.spotify.com/artist/6fEZjgt9MHR4Hp3MiBRZHX
123 (5). Jim Yosef https://open.spotify.com/artist/40HDiLfKm0tXk2FxlJx6aO
124 (5). Alex Skrindo https://open.spotify.com/artist/5LnNAR8Bhv2vJCJzBnCw9P
125 (5). Elektronomia https://open.spotify.com/artist/7qgorhDacy1FRz5HkAhDtu
126 (5). Ahrix https://open.spotify.com/artist/0kXCdaJ7C5MD27jXxzvHsa
127 (5). Ahxello https://open.spotify.com/artist/3avjTKhr5ZAxRV

228 (5). Jan Blomqvist https://open.spotify.com/artist/5wMlMjOLeJfS5DfxqGfm83
229 (5). Dirty Doering https://open.spotify.com/artist/4N6XVXvfjBPNFCdS56TCea
230 (5). AKA AKA https://open.spotify.com/artist/64fjAjykuM8Oc3Bqup4g72
231 (5). Monkey Safari https://open.spotify.com/artist/5zovXI5By2gUhdr7EByjLa
232 (5). Mollono.Bass https://open.spotify.com/artist/27j5PRcPefcI6q8as58zWF
233 (5). The Glitz https://open.spotify.com/artist/198itnIfOerunuIbXqrXr3
234 (5). Ron Flatter https://open.spotify.com/artist/0HZ3FXXgXyKoMHKwKSZOpq
235 (5). Oliver Schories https://open.spotify.com/artist/0iTjLBepeGaLgZS18kxgRq
236 (5). Super Flu https://open.spotify.com/artist/1iZiG82D4w7FLHvOUUj4zW
237 (5). Rampa https://open.spotify.com/artist/08jywfUS0hp8XYlYs0cvz8
238 (5). Ante Perry https://open.spotify.com/artist/4ueT3TFdSmR04bDAiZbjsW
239 (5). Umami https://open.spotify.com/artist/67qmoqrkr162LFix4Czmlv
240 (5). Stereo Express https://open.spotify.com/artist/3j2zB13syOvCyrkJIomEA2
241 (5). Joachim Pa

337 (5). Takaaki Itoh https://open.spotify.com/artist/2cE64RtSkS5CE0075eMMCt
338 (5). Shifted https://open.spotify.com/artist/3quiO6Un8G55TpLqs3gRv8
339 (5). Regis https://open.spotify.com/artist/4erwZm1JybuSPTUg5Bu7nT
340 (5). Developer https://open.spotify.com/artist/0f4ne0cd4tbiZCLzpb1QoX
341 (5). Bas Mooy https://open.spotify.com/artist/2w2yKSYPjPrgwDeDRq7npS
342 (5). Truncate https://open.spotify.com/artist/1JxLnEvBPdm9baryEFzE50
343 (5). Jeroen Search https://open.spotify.com/artist/2t9nXmwTyxsqEEpIdduEfj
344 (5). Planetary Assault Systems https://open.spotify.com/artist/7umQgFrDu3yrchEbFfJd60
345 (5). Rrose https://open.spotify.com/artist/5naKaYAyzzuPDsh4H2dwyT
346 (5). James Ruskin https://open.spotify.com/artist/1CxlOLe7rJ6EO7aPMxi9Xx
347 (5). Tensal https://open.spotify.com/artist/3mRdWhXS0ujP6WUjpOiHB1
348 (5). Perc https://open.spotify.com/artist/4iyqyvWXO734GUNLTTnrPQ
349 (5). Pfirter https://open.spotify.com/artist/6qavLbYvbSKWqwlz00CoBe
350 (5). Ø [Phase] https://open.sp

447 (5). DJ S.K.T https://open.spotify.com/artist/05z4P0nQO00HG5WeErugd0
448 (5). Illyus & Barrientos https://open.spotify.com/artist/6n5ccknvteaKfN4WA2qf3C
449 (5). Low Steppa https://open.spotify.com/artist/5OImcY3khBn9UFjzgaapob
450 (5). Patrick Topping https://open.spotify.com/artist/7yRimuQSC5Ks3T2Ts0iyZa
451 (5). Philip George https://open.spotify.com/artist/0Q9slhIaEgg190iG8udYIV
452 (5). Kideko https://open.spotify.com/artist/0ZwQMCRqfyh1OGQkBh9Cnj
453 (5). Solardo https://open.spotify.com/artist/0oO1IaDOBSeI96HbnCa5pZ
454 (5). Shift K3Y https://open.spotify.com/artist/26OrZl5U3VNGHU9qUj8EcM
455 (5). Sammy Porter https://open.spotify.com/artist/2D51qkOmTNsNQj3C4LIvH7
456 (5). Tough Love https://open.spotify.com/artist/16KSSLMXOdKQ2MHt9bOHTG
457 (5). Hannah Wants https://open.spotify.com/artist/7sK4hnuUOXw6VStDw0q8NI
458 (5). Ferreck Dawn https://open.spotify.com/artist/3cnAJv9gydgm52KFIsdvO8
459 (4). GotSome https://open.spotify.com/artist/5eALE6GKSAiBNMyqpsqoeX
460 (5). Gene F

558 (5). Prunk https://open.spotify.com/artist/6FJfLfGO9X2AVNz0sFscrG
559 (5). Toman https://open.spotify.com/artist/7A0eeETj5gjPjvXLnskbfG
560 (5). Dimmish https://open.spotify.com/artist/3BPvVhOECwv7HSHVZaq3BG
561 (5). Chris Stussy https://open.spotify.com/artist/3BxjasMelf9pKaE4f7Y0So
562 (5). Rossi. https://open.spotify.com/artist/7itMGcVGRKS43LcTQvJitf
563 (5). East End Dubs https://open.spotify.com/artist/5nw8Cj9x9XXKDuuNhl5mF7
564 (5). DJOKO https://open.spotify.com/artist/3fdC2UHP39tyWCUsHB6qlv
565 (5). Tommy Vercetti https://open.spotify.com/artist/2noNB4p7pYhP4XfNcdDfJc
566 (5). Blackchild (ITA) https://open.spotify.com/artist/0UVthdD1eqqsoNLX9ek4Xb
567 (5). Jesse Maas https://open.spotify.com/artist/2eprK9dqUOpKRzI18r7e8G
568 (5). Artmann https://open.spotify.com/artist/2p0rSIjHFpx3IYYenqB14y
